In [ ]:
import os
import pandas as pd
from econ_evals.utils.helper_functions import get_base_dir_path
import statistics

from econ_evals.experiments.patience_vs_impatience.helper_functions import (
    calc_future_value,
    calc_reliability_score,
    horizon_str_to_horizon_length,
    detect_horizon_str,
)

# Load data

In [ ]:
log_dirnames = os.listdir(
    get_base_dir_path() / "experiments/patience_vs_impatience/logs"
)

# Contains directory name and time horizon (in years) as key-value pairs
dirname_to_horizon_length = {
    get_base_dir_path()
    / "experiments/patience_vs_impatience/logs"
    / dirname: horizon_str_to_horizon_length(detect_horizon_str(dirname))
    for dirname in log_dirnames
}

MODELS = ["claude-3-5-sonnet-20241022", "gpt-4o-2024-11-20", "gemini-1.5-pro-002"]

num_trials = 20

# Calculate reliability and litmus scores for each time horizon

In [ ]:
interest_rates = [1 + 0.001 * i for i in range(2000)]
for dirname, timelength in dirname_to_horizon_length.items():
    df = pd.read_csv(dirname / "high_level_results.csv")
    # Sort data by 'value' so the line plot is in ascending order
    df = df.sort_values(by="value")
    # Plot the number of times the model chose the delayed reward
    df["future_reward"] = num_trials - df["preferred_100_count"]
    # Normalize the counts by the number of trials
    df["future_reward"] = df["future_reward"] / num_trials
    for model in MODELS:
        model_df = df[df["model"] == model].copy()
        reliability_scores = []
        for interest_rate in interest_rates:
            converted_interest_rate_value = calc_future_value(interest_rate, timelength)
            reliability_scores.append(
                calc_reliability_score(model_df, converted_interest_rate_value)
            )
        max_value = max(reliability_scores)
        indices = [
            i for i, value in enumerate(reliability_scores) if value == max_value
        ]
        calc_interest_rate = [interest_rates[i] for i in indices]
        calc_interest_rate = statistics.median(calc_interest_rate)
        interest_rate = calc_interest_rate * 100 - 100
        print(
            f"Model: {model} with timeframe: {timelength:.3f}y, Max reliability score: {max_value:.3f}, Interest rates: {interest_rate:.2f}"
        )
    print()

# Calculate reliability and litmus scores aggregated over all time horizons

In [ ]:
interest_rates = [1 + 0.001 * i for i in range(201)]
data_records = []
for model in MODELS:
    reliability_scores = []
    for interest_rate in interest_rates:
        reliability_scores_for_specific_interest_rate = []
        for dirname, timelength in dirname_to_horizon_length.items():
            converted_interest_rate_value = calc_future_value(interest_rate, timelength)
            df = pd.read_csv(dirname / "high_level_results.csv")
            df = df.sort_values(by="value")
            df["future_reward"] = num_trials - df["preferred_100_count"]
            df["future_reward"] = df["future_reward"] / num_trials
            model_df = df[df["model"] == model].copy()
            reliability_scores_for_specific_interest_rate.append(
                calc_reliability_score(model_df, converted_interest_rate_value)
            )
        # Aggregate the reliability scores for the different time lengths
        reliability_score = sum(reliability_scores_for_specific_interest_rate) / len(
            reliability_scores_for_specific_interest_rate
        )
        reliability_scores.append(reliability_score)
        data_records.append(
            {
                "model": model,
                "interest_rate": interest_rate,
                "reliability_score": reliability_score,
            }
        )
    max_value = max(reliability_scores)
    indices = [i for i, value in enumerate(reliability_scores) if value == max_value]
    calc_interest_rate = [interest_rates[i] for i in indices]
    calc_interest_rate = statistics.median(calc_interest_rate)
    interest_rate = calc_interest_rate * 100 - 100
    print(f"Model: {model}")
    print(f"Interest rate: {interest_rate:.2f}")
    print(f"Reliability scores: {max_value}")
    print()
df = pd.DataFrame(data_records)